# Adding robust image geometry feature

Currently donuts image geometry isn't robust and fails for certain CCD sizes. 
Here we prototype some robust geometry selection.

Reported issue:
```Dear James, I'm using Donuts to implement a guiding procedure for a small telescope, occasionally I get an error at Donuts initialization:

 File "/home/lfini/tmp/Donuts/donuts/image.py", line 172, in remove_background
    self.backsub_region = self.raw_region - self.sky_background
ValueError: operands could not be broadcast together with shapes (2928,2928) (2912,2912)

This happens with the following call:

don = donuts.Donuts(refimage=image, image_ext=0, # overscan_width=20, prescan_width=20,
                    border=64, normalise=True, exposure='EXPOSURE', subtract_bkg=True, ntiles=32)

on an Image 3056x3056 pixels wide

I can avoid the error if I uncomment the overscan and prescan arguments, setting both equal to 20.

Looking at the code I believe it's a problem depending on the fact that the image size, after removing the border, is not an integral muiltiple of number of tiles (in my case 3056-128 = 2928i with ntiles=32)

My question is:  is Donuts supposed to cope with different combinations of image sizes/border/prescan/overscan values and I've discovered a bug, or its me who must compute proper values of the arguments depending on the image size? And in the latter case how are them to be computed?

Many thanks,
                                Luca
```

Proposed solution:
   1. Rework the image geometry code to accept images of all shapes
   1. Do as requested (using available parameters)
   1. Or at least fall back to a sensible approximation of the request if the geometry requested doesn't work. 
   
The problem comes from the fact that the image background subtraction needs the image to be equally divisible into a series of tiles. This error can be seen above. On my limited testing this feature worked fine, but is clearly falling over under some circumstances. 


Current steps for trimming an image:
   1. Apply a pre/overscan correction, if required
   1. "Check if the CCD is a funny shape" - this bit seems very odd and is likely the cause of our problems
   1. Apply the border calculation to what is left of the previous step
   1. Image normalisation, if required
   1. Background subtraction, if required
   
What I'd like to happen:
   1. Apply the pre/overscan correction, if required
   1. Apply the border trimming
   1. SANITY CHECK THE REMAINING IMAGE GEOMETRY AND THE NUMBER OF TILES REQUESTED WORK TOGETHER
      1. IF NOT, TWEAK SLIGHTLY THE IMAGE GEOMETRY TO SLICE AN INTEGER NUMBER OF TILES IN BOTH AXES
   1. Store the final image geometry so we only have to work this out once per reference image
   1. Image normalisation, if required
   1. Background subtraction, if required
   1. For all subsequent images use the stored image geometry
   
There was an issue/PR submitted a long time ago that requested handling user supplied image sub-sections. Given donuts has been living in the wild for a while now I don't want to introduce non-backwards compatible changes. Therefore to handle this feature request we can do the following:
   1. Add a new switch to donuts to call for manually supplied geometry in the form of a list/tuple
   1. If this list is supplied we then:
      1. Skip pre/overscan correction
      1. Skip the border correction
      1. Continue from application of normalisation, if required
      1. Perform background subtraction, if required
      


In [ ]:
# Offending image method and the background subtraction method, below we modify trim() to fix it and run some tests

def trim(self, prescan_width=0, overscan_width=0,                             
         scan_direction='x', border=64):                                      
    '''Remove the optional prescan and overscan from the image, as well       
    as the outer `n` rows/colums of the image. Finally ensure the imaging     
    region is the correct dimensions for :py:func:`skimage.transform.resize`  
    (i.e. a multiple of 16.)                                                  
                                                                              
    Parameters                                                                
    ----------                                                                
    prescan_width : int                                                       
        Remove the first ``prescan_width`` columns from the image, assuming   
        the are not in the imaging region.                                    
                                                                              
    overscan_width : int                                                      
        Remove the last ``overscan_width`` columns from the image, assuming   
        the are not in the imaging region.                                    
                                                                              
    scan_direction : 'x' | 'y'                                                
        Direction along which the pre/overscans occur. If along left and right
        side, select 'x'. If along the top and bottom of the image select 'y' 
                                                                              
    border : int                                                              
        Ignore the first/last ``border`` rows/columns from the image,         
        assuming that they are not "typical", a common case with edge         
        effects in CCDs.                                                      
                                                                              
    Returns                                                                   
    -------                                                                   
    self : :class:`~donuts.image.Image`                                       
        The current :class:`~donuts.image.Image` instance                     
                                                                              
    Raises                                                                    
    ------                                                                    
    None                                                                      
                                                                              
                                                                              
    '''                                                                       
    if overscan_width > 0 and prescan_width > 0:                             
        if scan_direction == 'x':                                            
            image_section = self.raw_image[:, prescan_width:-overscan_width] 
        else:                                                                
            image_section = self.raw_image[prescan_width:-overscan_width, :] 
    elif overscan_width > 0:                                                 
        if scan_direction == 'x':                                            
            image_section = self.raw_image[:, :-overscan_width]              
        else:                                                                
            image_section = self.raw_image[:-overscan_width, :]              
    elif prescan_width > 0:                                                  
        if scan_direction == 'x':                                            
            image_section = self.raw_image[:, prescan_width:]                
        else:                                                                
            image_section = self.raw_image[prescan_width:, :]                
    else:                                                                    
        image_section = self.raw_image                                       

    dy, dx = image_section.shape                                             

    # check if the CCD is a funny shape. Normal CCDs should divide by 16     
    # with no remainder. NITES for example does not (1030,1057)              
    # instead of (1024,1024)                                                 
    rx = dx % 16                                                             
    ry = dy % 16                                                             
    base = 512                                                               

    if rx > 0 or ry > 0:                                                     
        dimx = int(dx // base) * base                                        
        dimy = int(dy // base) * base                                        
    else:                                                                    
        dimx = dx                                                            
        dimy = dy                                                            

    # get the reference data, with tweaked shape if needed                   
    self.raw_region = image_section[                                         
        border:dimy - border, border:dimx - border                           
    ]                                                                        
    return self                                                              


def remove_background(self, ntiles=32):                         
    '''Subtract the background from the image. See              
    :py:meth:`~donuts.image.Image._generate_bkg_map` for        
    more details                                                
                                                                
    Parameters                                                  
    ----------                                                  
    ntiles : int                                                
        Number of tiles used to sample the sky background.      
        The default is 32.                                      
                                                                
    Returns                                                     
    -------                                                     
    self : :class:`~donuts.image.Image`                         
        The current :class:`~donuts.image.Image` instance       
                                                                
                                                                
    Raises                                                      
    ------                                                      
    None                                                        
    '''                                                         
    dim_y, dim_x = self.raw_region.shape                        
    tilesize_x, tilesize_y = dim_x // ntiles, dim_y // ntiles   
    self.sky_background = self._generate_bkg_map(               
        data=self.raw_region,                                   
        tile_num=ntiles,                                        
        tilesizex=tilesize_x,                                   
        tilesizey=tilesize_y                                    
    )                                                           
    self.backsub_region = self.raw_region - self.sky_background 
    return self                                                 

In [ ]:
# Imports
import numpy as np
from skimage.transform import resize 

In [ ]:
def trim_test(raw_image, prescan_width=0, overscan_width=0,                             
              scan_direction='x', border=64):       
    """
    Modified to take in raw_image and return raw_region (without self ofc)
    """
    if overscan_width > 0 and prescan_width > 0:                             
        if scan_direction == 'x':                                            
            image_section = raw_image[:, prescan_width:-overscan_width] 
        else:                                                                
            image_section = raw_image[prescan_width:-overscan_width, :] 
    elif overscan_width > 0:                                                 
        if scan_direction == 'x':                                            
            image_section = raw_image[:, :-overscan_width]              
        else:                                                                
            image_section = raw_image[:-overscan_width, :]              
    elif prescan_width > 0:                                                  
        if scan_direction == 'x':                                            
            image_section = raw_image[:, prescan_width:]                
        else:                                                                
            image_section = raw_image[prescan_width:, :]                
    else:                                                                    
        image_section = raw_image                                       

    dy, dx = image_section.shape                                             

    # check if the CCD is a funny shape. Normal CCDs should divide by 16     
    # with no remainder. NITES for example does not (1030,1057)              
    # instead of (1024,1024)                                                 
    rx = dx % 16                                                             
    ry = dy % 16                                                             
    base = 512                                                               

    if rx > 0 or ry > 0:                                                     
        dimx = int(dx // base) * base                                        
        dimy = int(dy // base) * base                                        
    else:                                                                    
        dimx = dx                                                            
        dimy = dy                                                            

    # get the reference data, with tweaked shape if needed                   
    raw_region = image_section[                                         
        border:dimy - border, border:dimx - border                           
    ]                                                                        
    return raw_region  

def _generate_bkg_map(data, tile_num, tilesizex, tilesizey):                 
    """
    Modifed to remove self
    """                                                                        
    # create coarse map                                                            
    coarse = np.empty((tile_num, tile_num))                                        
    for i in range(0, tile_num):                                                   
        for j in range(0, tile_num):                                               
            coarse[i][j] = np.median(data[(i * tilesizey):(i + 1) * tilesizey,     
                                          (j * tilesizex):(j + 1) * tilesizex])    
    # resample it out to data size                                                 
    try:                                                                           
        bkgmap = resize(                                                           
            coarse,                                                                
            (tilesizey * tile_num, tilesizex * tile_num),                          
            mode='edge'                                                            
        )                                                                          
    except ValueError:                                                             
        # "edge" mode is not supported on the current version of                   
        # scikit-image                                                             
        bkgmap = resize(                                                           
            coarse,                                                                
            (tilesizey * tile_num, tilesizex * tile_num),                          
            mode='nearest'                                                         
        )                                                                          
                                                                                   
    return bkgmap                                                                  


def remove_background_test(raw_region, ntiles=32):                         
    """
    Modifed to remove self
    """                                                    
    dim_y, dim_x = raw_region.shape                        
    tilesize_x, tilesize_y = dim_x // ntiles, dim_y // ntiles   
    sky_background = _generate_bkg_map(               
        data=raw_region,                                   
        tile_num=ntiles,                                        
        tilesizex=tilesize_x,                                   
        tilesizey=tilesize_y                                    
    )                                                           
    backsub_region = raw_region - sky_background 
    return backsub_region

In [ ]:
# replicate Lucas's issue
img = np.ones((3056, 3056))
print(img.shape)

# trim the image
raw_region = trim_test(img, prescan_width=0, overscan_width=0, border=64)
print(raw_region.shape)

# remove the background
backsub_region = remove_background_test(raw_region, ntiles=32)

In [ ]:
def trim_new(raw_image, prescan_width=0, overscan_width=0,                             
             scan_direction='x', border=64, ntiles=32):       
    """
    Fixed trim
    
    Want to keep track of image section corners so we can do the calculation once
    then just apply it later
    """
    
    # keep track of image corners so we only have to do this calculation once
    # per reference image
    fy, fx = raw_image.shape
    cly, cuy = 0, fy
    clx, cux = 0, fx
    
    if overscan_width > 0 and prescan_width > 0:                             
        if scan_direction == 'x':                                            
            image_section = raw_image[:, prescan_width:-overscan_width] 
            clx = prescan_width
            cux = fx-overscan_width
        else:                                                                
            image_section = raw_image[prescan_width:-overscan_width, :] 
            cly = prescan_width
            cuy = fy-overscan_width
    elif overscan_width > 0:                                                 
        if scan_direction == 'x':                                            
            image_section = raw_image[:, :-overscan_width] 
            cux = fx-overscan_width
        else:                                                                
            image_section = raw_image[:-overscan_width, :]   
            cuy = fy-overscan_width
    elif prescan_width > 0:                                                  
        if scan_direction == 'x':                                            
            image_section = raw_image[:, prescan_width:]  
            clx = prescan_width 
        else:                                                                
            image_section = raw_image[prescan_width:, :]        
            cly = prescan_width
    else:                                                                    
        image_section = raw_image    
    
    print("----")
    print(cly, cuy, clx, cux)
    print(f"size track: {cuy - cly}, {cux - clx}")
    print(image_section.shape)
    print("----")
    
    #dy, dx = image_section.shape                                             
    #
    # check if the CCD is a funny shape. Normal CCDs should divide by 16     
    # with no remainder. NITES for example does not (1030,1057)              
    # instead of (1024,1024)                                                 
    #rx = dx % 16                                                             
    #ry = dy % 16                                                             
    #base = 512                                                               
    #
    #if rx > 0 or ry > 0:                                                     
    #    dimx = int(dx // base) * base                                        
    #    dimy = int(dy // base) * base                                        
    #else:                                                                    
    #    dimx = dx                                                            
    #    dimy = dy                                                            
    
    # remove a border if required
    if border > 0:
        cly += border
        cuy -= border
        clx += border
        cux -= border
        raw_region = image_section[                                         
            border:-border, border:-border                           
        ] 
    else:
        raw_region = image_section
        
    print("----")
    print(cly, cuy, clx, cux)
    print(f"size track: {cuy - cly}, {cux - clx}")
    print(raw_region.shape)
    print("----")
    
    # check we have an image section that is equally divisible by ntiles,
    # apply final tweak if not.
    ry, rx = raw_region.shape
    trim_y = ry % ntiles
    trim_x = rx % ntiles
    
    # trim y
    if trim_y > 0:
        print(f"Warning, removing y={trim_y} pixels from image upper edge")
        raw_region = raw_region[:-trim_y, :]
        cuy -= trim_y
    if trim_x > 0:
        print(f"Warning, removing x={trim_x} pixels from image right edge")
        raw_region = raw_region[:, :-trim_x]
        cux -= trim_x
        
    print("----")
    print(cly, cuy, clx, cux)
    print(f"size track: {cuy - cly}, {cux - clx}")
    print(raw_region.shape)
    print("----")        
    
    return raw_region  

def _generate_bkg_map(data, tile_num, tilesizex, tilesizey):                 
    """
    Modifed to remove self
    """                                                                        
    # create coarse map                                                            
    coarse = np.empty((tile_num, tile_num))                                        
    for i in range(0, tile_num):                                                   
        for j in range(0, tile_num):                                               
            coarse[i][j] = np.median(data[(i * tilesizey):(i + 1) * tilesizey,     
                                          (j * tilesizex):(j + 1) * tilesizex])    
    # resample it out to data size                                                 
    try:                                                                           
        bkgmap = resize(                                                           
            coarse,                                                                
            (tilesizey * tile_num, tilesizex * tile_num),                          
            mode='edge'                                                            
        )                                                                          
    except ValueError:                                                             
        # "edge" mode is not supported on the current version of                   
        # scikit-image                                                             
        bkgmap = resize(                                                           
            coarse,                                                                
            (tilesizey * tile_num, tilesizex * tile_num),                          
            mode='nearest'                                                         
        )                                                                          
                                                                                   
    return bkgmap                                                                  


def remove_background_new(raw_region, ntiles=32):                         
    """
    Modifed to remove self
    """                                                    
    dim_y, dim_x = raw_region.shape                        
    tilesize_x, tilesize_y = dim_x // ntiles, dim_y // ntiles   
    sky_background = _generate_bkg_map(               
        data=raw_region,                                   
        tile_num=ntiles,                                        
        tilesizex=tilesize_x,                                   
        tilesizey=tilesize_y                                    
    )                                                           
    backsub_region = raw_region - sky_background 
    return backsub_region

In [ ]:
# replicate Lucas's issue with fixed code
img = np.ones((3056, 3056))

# trim the image
raw_region = trim_new(img, prescan_width=0, overscan_width=0, border=64)

# remove the background
backsub_region = remove_background_new(raw_region, ntiles=32)

<h1 style='color:green'>PASS</h1>

<h3>Now make a tonne of image shapes and test this code works</h3>

In [ ]:
# simulate a bunch of image shapes with different o/pscans and borders
# and look for failures
oscans = [0, ] #10, 20, 30]
pscans = [20, ] # 10, 0, 30]
borders = [0, ] # 12, 40, 64]

for oscan, pscan, border in zip(oscans, pscans, borders):
    randx = np.random.randint(500, 10000, 50)
    randy = np.random.randint(500, 10000, 50)
    for i, j in zip(randx, randy):
        # simulate some random geometry
        img = np.ones((i, j))
        print(f"\n\n\n")

        # trim the image
        raw_region = trim_new(img, prescan_width=oscan, overscan_width=pscan, border=border)

        # remove the background
        backsub_region = remove_background_new(raw_region, ntiles=32)